# 03. RSS 스크레이핑하기

## 라이브러리 설치하기

### feedparser
- RSS 1.0, RSS 2.0, Atom 등 여러 사양에 모두 대응

In [1]:
import feedparser

feedparser.__version__

'6.0.8'

### XML 분석하기
- 기상청 동네 예보 RSS 사용(https://www.weather.go.kr/w/pop/rss-guide.do)
- parse 메서드에 URL 지정하여 호출

In [2]:
import feedparser

rss = feedparser.parse('http://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=1162060500')
print(rss)

{'bozo': False, 'entries': [{'authors': [{'name': '기상청'}], 'author': '기상청', 'author_detail': {'name': '기상청'}, 'tags': [{'term': '서울특별시 관악구 은천동', 'scheme': None, 'label': None}], 'title': '동네예보(도표) : 서울특별시 관악구 은천동 [X=59,Y=125]', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'http://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=1162060500', 'value': '동네예보(도표) : 서울특별시 관악구 은천동 [X=59,Y=125]'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'http://www.kma.go.kr/weather/forecast/timeseries.jsp?searchType=INTEREST&dongCode=1162060500'}], 'link': 'http://www.kma.go.kr/weather/forecast/timeseries.jsp?searchType=INTEREST&dongCode=1162060500', 'id': 'http://www.kma.go.kr/weather/forecast/timeseries.jsp?searchType=INTEREST&dongCode=1162060500', 'guidislink': False, 'summary': '<header>\n  202110021700\n  5\n  59\n  125\n </header>\n \n  \n   21\n   0\n   20.0\n   -999.0\n   -999.0\n   1\n   0\n   맑음\n   Clear\n   0\n   0.0\n   0.0\n   0.9\n   6\n   서\n   W\n   80\n   0.0\n

In [3]:
rss['entries'][0]['summary']

'<header>\n  202110021700\n  5\n  59\n  125\n </header>\n \n  \n   21\n   0\n   20.0\n   -999.0\n   -999.0\n   1\n   0\n   맑음\n   Clear\n   0\n   0.0\n   0.0\n   0.9\n   6\n   서\n   W\n   80\n   0.0\n   0.0\n  \n  \n   24\n   0\n   20.0\n   -999.0\n   -999.0\n   1\n   0\n   맑음\n   Clear\n   0\n   0.0\n   0.0\n   1.0\n   4\n   남\n   S\n   85\n   0.0\n   0.0\n  \n  \n   3\n   1\n   19.0\n   29.0\n   18.0\n   1\n   0\n   맑음\n   Clear\n   0\n   0.0\n   0.0\n   1.8\n   3\n   남동\n   SE\n   90\n   0.0\n   0.0\n  \n  \n   6\n   1\n   19.0\n   29.0\n   18.0\n   1\n   0\n   맑음\n   Clear\n   0\n   0.0\n   0.0\n   1.7000000000000002\n   3\n   남동\n   SE\n   85\n   0.0\n   0.0\n  \n  \n   9\n   1\n   22.0\n   29.0\n   18.0\n   1\n   0\n   맑음\n   Clear\n   0\n   0.0\n   0.0\n   2.3000000000000003\n   4\n   남\n   S\n   75\n   0.0\n   0.0\n  \n  \n   12\n   1\n   26.0\n   29.0\n   18.0\n   1\n   0\n   맑음\n   Clear\n   0\n   0.0\n   0.0\n   3.2\n   5\n   남서\n   SW\n   60\n   0.0\n   0.0\n  \n  \n   15\n

### BeautifulSoup을 이용
- 기상청 RSS의 경우 feedparser보다 BeautifulSoup을 이용하는 것이 더 좋다.

In [4]:
import urllib.request 
import urllib.parse as parse
from bs4 import BeautifulSoup

rssurl = 'http://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=1162060500'

html = urllib.request.urlopen(rssurl).read()
text = html.decode("utf-8")
soup = BeautifulSoup(text,'html.parser')

In [5]:
soup.find_all('title')

[<title>기상청 동네예보 웹서비스 - 서울특별시 관악구 은천동 도표예보</title>,
 <title>동네예보(도표) : 서울특별시 관악구 은천동 [X=59,Y=125]</title>]

In [6]:
data = soup.find_all('data')
len(data)

18

In [7]:
data[0].temp.string

'20.0'

In [8]:
for d in data:
    time = int(d.hour.string)
    day = d.day.string
    temp = d.temp.string
    tmx = d.tmx.string
    tmn = d.tmn.string
    wf = d.wfkor.string
    pop = d.pop.string
    
    daydict = {'0':'오늘', '1':'내일', '2':'모레'}
    print(f'{daydict[day]} {time:02d}시의 날씨 : 기온 {temp}도, 최고기온 {tmx}도, 최저기온 {tmn}도, {wf}, 강수 확률 {pop}%')

오늘 21시의 날씨 : 기온 20.0도, 최고기온 -999.0도, 최저기온 -999.0도, 맑음, 강수 확률 0%
오늘 24시의 날씨 : 기온 20.0도, 최고기온 -999.0도, 최저기온 -999.0도, 맑음, 강수 확률 0%
내일 03시의 날씨 : 기온 19.0도, 최고기온 29.0도, 최저기온 18.0도, 맑음, 강수 확률 0%
내일 06시의 날씨 : 기온 19.0도, 최고기온 29.0도, 최저기온 18.0도, 맑음, 강수 확률 0%
내일 09시의 날씨 : 기온 22.0도, 최고기온 29.0도, 최저기온 18.0도, 맑음, 강수 확률 0%
내일 12시의 날씨 : 기온 26.0도, 최고기온 29.0도, 최저기온 18.0도, 맑음, 강수 확률 0%
내일 15시의 날씨 : 기온 28.0도, 최고기온 29.0도, 최저기온 18.0도, 맑음, 강수 확률 0%
내일 18시의 날씨 : 기온 25.0도, 최고기온 29.0도, 최저기온 18.0도, 맑음, 강수 확률 0%
내일 21시의 날씨 : 기온 23.0도, 최고기온 29.0도, 최저기온 18.0도, 구름 많음, 강수 확률 20%
내일 24시의 날씨 : 기온 22.0도, 최고기온 29.0도, 최저기온 18.0도, 흐림, 강수 확률 30%
모레 03시의 날씨 : 기온 21.0도, 최고기온 26.0도, 최저기온 19.0도, 구름 많음, 강수 확률 20%
모레 06시의 날씨 : 기온 20.0도, 최고기온 26.0도, 최저기온 19.0도, 흐림, 강수 확률 30%
모레 09시의 날씨 : 기온 22.0도, 최고기온 26.0도, 최저기온 19.0도, 흐림, 강수 확률 30%
모레 12시의 날씨 : 기온 25.0도, 최고기온 26.0도, 최저기온 19.0도, 흐림, 강수 확률 30%
모레 15시의 날씨 : 기온 26.0도, 최고기온 26.0도, 최저기온 19.0도, 흐림, 강수 확률 30%
모레 18시의 날씨 : 기온 24.0도, 최고기온 26.0도, 최저기온 19.0도, 흐림, 강수 확률 30%
모레 21시의 날씨 : 기온 23